In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
import os
model = ChatGoogleGenerativeAI(
                model='gemini-2.0-flash',
                google_api_key=os.getenv('GOOGLE_API_KEY'),
                temperature=0,
                max_output_tokens=1000
            )

model.invoke('Hello')

In [ ]:
https://github.com/campusx-official/langgraph-tutorials/blob/main/7_review_reply_workflow.ipynb